# Import data

In [ ]:
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path Data/raw_data/ \
    --input-format CasavaOneEightSingleLanePerSampleDirFmt \
    --output-path Data/demux-paired-end.qza

In [ ]:
!qiime demux summarize \
    --i-data Data/demux-paired-end.qza \
    --o-visualization Results/demux-paired-end.qzv 

# Use cutadapt to remove primers

In [ ]:
!qiime cutadapt trim-paired \
    --i-demultiplexed-sequences Data/demux-paired-end.qza \
    --o-trimmed-sequences Data/trimmed-paired-end.qza \
    --p-cores 18 \
    --p-front-f AGATGTGTATAAGAGACAGCCTACGGGNGGCNGCAG \
    --p-front-r GACTACNNGGGTATCTAATCC \
    --p-match-adapter-wildcards
# Visualization
!mkdir Results
!qiime demux summarize \
    --i-data Data/trimmed-paired-end.qza \
    --o-visualization Results/trimmed-paired-end.qzv 

# Denoising with DADA2
Note. We modified R script for Dada2 (file "anaconda3/envs/qiime2-2019.4/bin/run_dada_paired.R") by providing “mergers[[j]] <- mergePairs(ddF, drpF, ddR, drpR, minOverlap=4)” to allow change minimum overlap to 4

In [ ]:
!mkdir Data/tables
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs Data/trimmed-paired-end.qza \
    --p-trim-left-f 2 \
    --p-trim-left-r 2 \
    --p-trunc-len-f 267 \
    --p-trunc-len-r 238 \
    --p-n-threads 18 \
    --o-table Data/tables/table.qza \
    --o-representative-sequences Data/rep-seqs.qza \
    --o-denoising-stats Data/denoising-stats.qza

!qiime feature-table tabulate-seqs \
    --i-data Data/rep-seqs.qza \
    --o-visualization Results/rep-seqs.qzv

!qiime metadata tabulate \
    --m-input-file Data/denoising-stats.qza \
    --o-visualization Results/denoising-stats.qzv

!mkdir -p Results/tables
!qiime feature-table summarize \
    --i-table Data/tables/table.qza \
    --o-visualization Results/tables/table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

!rm Data/demux-paired-end.qza Data/trimmed-paired-end.qza

# Taxonomic analysis

In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-classifier training-feature-classifiers/classifier.qza \
    --i-reads Data/rep-seqs.qza \
    --o-classification Data/taxonomy.qza
    #--p-reads-per-batch 20000 \

!qiime tools export \
    --input-path Data/taxonomy.qza \
    --output-path Data/taxonomy-with-spaces
!qiime metadata tabulate \
    --m-input-file Data/taxonomy-with-spaces/taxonomy.tsv  \
    --o-visualization Data/taxonomy-as-metadata.qzv
!qiime tools export \
    --input-path Data/taxonomy-as-metadata.qzv \
    --output-path Data/taxonomy-as-metadata
!qiime tools import \
    --type 'FeatureData[Taxonomy]' \
    --input-path Data/taxonomy-as-metadata/metadata.tsv \
    --output-path Data/taxonomy-without-spaces.qza
!qiime metadata tabulate \
    --m-input-file Data/taxonomy-without-spaces.qza \
    --o-visualization Results/taxonomy-without-spaces.qzv
!rm -r Data/taxonomy-with-spaces Data/taxonomy-as-metadata

# BIOM editing to add taxa to ASVs hashes in BIOM table

In [8]:
!conda install -y -c anaconda biopython

Solving environment: done

## Package Plan ##

  environment location: /home/timyerg/anaconda3/envs/qiime2-2020.2

  added / updated specs:
    - biopython


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.76             |   py36h7b6447c_0         2.6 MB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.6 MB

The following NEW packages will be INSTALLED:

  biopython          anaconda/linux-64::biopython-1.76-py36h7b6447c_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> anaconda::ca-certificates-2020.1.1-0
  certifi            conda-forge::certifi-2020.4.5

In [4]:
#input
table    = 'Data/tables/table.qza'
taxa     = 'Data/taxonomy-without-spaces.qza'
rep_seq  = 'Data/rep-seqs.qza'

#import
import pandas as pd
from Bio import SeqIO

#export rep-seqs.qza, table.qza and taxonomy.qza
!mkdir Biom Taxa Rep-seqs

!qiime tools export \
    --input-path $table \
    --output-path Biom/
!biom convert -i Biom/feature-table.biom -o Biom/feature-table.tsv --to-tsv

!qiime tools export \
    --input-path $taxa \
    --output-path Taxa

!qiime tools export \
    --input-path $rep_seq \
    --output-path Rep-seqs/

#replacing hashes with combination of taxonomy and beginings of the hashes
taxa_df = pd.read_csv('Taxa/taxonomy.tsv', sep='\t', skiprows=[1])
biom_df = pd.read_csv('Biom/feature-table.tsv', sep='\t', skiprows=1)

taxa_df['Combo'] = pd.DataFrame(taxa_df['Taxon'].astype(str).str.replace(';__', '').str.replace('[', '').str.replace(']', '')
                                .str.replace('.', '').str.replace('/', '_').str.replace("'", '').str.replace(' ', '_')
                                .str.split("__").str[-1].str.split(";").str[-1])
for n in range(2,7):
    taxa_df.loc[taxa_df['Combo'].str.contains('uncultured_bacterium'),'Combo'] = taxa_df['Taxon'].astype(str)\
        .str.replace(';__','').str.replace('[','').str.replace(']','').str.replace('.','').str.replace('/','_')\
        .str.replace("'",'').str.replace(' ','_').str.split("__").str[-n].str.split(';').str[0]+'_unc_b'
    taxa_df.loc[taxa_df['Combo'].str.contains('uncultured'),'Combo'] = taxa_df['Taxon'].astype(str)\
        .str.replace(';__','').str.replace('[','').str.replace(']','').str.replace('.','').str.replace('/','_')\
        .str.replace("'",'').str.replace(' ','_').str.split("__").str[-n].str.split(';').str[0]+'_unc'



biom_df['#OTU ID'] = taxa_df['Combo']+'|'+taxa_df['Feature ID']
taxa_df['Feature ID'] = biom_df['#OTU ID']
taxa_df = taxa_df[['Feature ID', 'Taxon', 'Confidence']]

#writing all files back
biom_df.to_csv('Biom/feature-table.tsv', sep='\t', index=False)
taxa_df.to_csv('Taxa/taxonomy.tsv', sep='\t', index=False)

fasta_hash  = r"Rep-seqs/dna-sequences.fasta"
fasta_combo = r"Rep-seqs/dna-sequences.fa"
i=0
with open(fasta_hash) as hashes, open(fasta_combo, 'w') as combo:
    records = SeqIO.parse(fasta_hash, 'fasta')
    for record in records:
        record.id = biom_df['#OTU ID'][i]
        record.description = ''
        i+=1
        SeqIO.write(record, combo, 'fasta')
!rm $fasta_hash
!mv $fasta_combo $fasta_hash

Exported Data/tables/table.qza as BIOMV210DirFmt to directory Biom/
Exported Data/taxonomy-without-spaces.qza as TSVTaxonomyDirectoryFormat to directory Taxa
Exported Data/rep-seqs.qza as DNASequencesDirectoryFormat to directory Rep-seqs/


In [3]:
#creating new rep-seqs.qza, table.qza and taxonomy.qza with modified hashes
!biom convert -i Biom/feature-table.tsv -o Biom/feature-table.biom --table-type="OTU table" --to-hdf5
!qiime tools import \
    --input-path Biom/feature-table.biom \
    --type 'FeatureTable[Frequency]' \
    --input-format BIOMV210Format \
    --output-path Data/tables/combo_table.qza
!qiime tools import \
    --type 'FeatureData[Taxonomy]' \
    --input-path Taxa/taxonomy.tsv \
    --output-path Data/combo_taxonomy.qza
!qiime tools import \
    --input-path $fasta_hash \
    --output-path Data/combo_rep-seqs.qza \
    --type 'FeatureData[Sequence]'

!rm -r Biom Taxa Rep-seqs

Imported Biom/feature-table.biom as BIOMV210Format to Data/tables/combo_table.qza
Imported Taxa/taxonomy.tsv as TSVTaxonomyDirectoryFormat to Data/combo_taxonomy.qza
Imported Rep-seqs/dna-sequences.fasta as DNASequencesDirectoryFormat to Data/combo_rep-seqs.qza


# Generate a tree for phylogenetic diversity analysis

In [1]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences Data/combo_rep-seqs.qza \
    --o-alignment Data/aligned-rep-seqs.qza \
    --o-masked-alignment Data/masked-aligned-rep-seqs.qza \
    --o-tree Data/unrooted-tree.qza \
    --o-rooted-tree Data/rooted-tree.qza \
    --p-n-threads 18

Saved FeatureData[AlignedSequence] to: Data/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: Data/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: Data/unrooted-tree.qza
Saved Phylogeny[Rooted] to: Data/rooted-tree.qza


# Filtration to remove low abundant features

In [3]:
!qiime feature-table filter-features \
    --i-table Data/tables/combo_table.qza \
    --p-min-frequency 10 \
    --p-min-samples 5 \
    --o-filtered-table Data/tables/sample-contingency-filtered-table.qza

!qiime taxa filter-table \
    --i-table Data/tables/sample-contingency-filtered-table.qza \
    --i-taxonomy Data/combo_taxonomy.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table Data/tables/table-no-mit_chlor.qza

!qiime feature-table filter-samples \
    --i-table Data/tables/table-no-mit_chlor.qza \
    --p-min-features 25 \
    --p-min-frequency 8000 \
    --o-filtered-table Data/tables/filtered-table.qza

!qiime feature-table filter-samples \
    --i-table Data/tables/filtered-table.qza \
    --m-metadata-file Data/tables/samples_to_exclude.tsv \
    --p-exclude-ids \
    --o-filtered-table Data/tables/filtered-table.qza

!mkdir Results/tables
!qiime feature-table summarize \
    --i-table Data/tables/filtered-table.qza \
    --o-visualization Results/tables/filtered-table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

Saved FeatureTable[Frequency] to: Data/tables/table-no-mit_chlor.qza
Saved FeatureTable[Frequency] to: Data/tables/filtered-table.qza
Saved FeatureTable[Frequency] to: Data/tables/filtered-table.qza
Saved Visualization to: Results/tables/filtered-table.qzv


# Excluding unassigned and assigned only to bacteria level features
Extracted manually a taxonomy.tsv file from combo_taxonomy.qza and filtered it to leave only features to exclude.

In [5]:
!qiime feature-table filter-features \
    --i-table Data/tables/filtered-table.qza \
    --m-metadata-file Data/tables/taxonomy_to_exclude.tsv \
    --p-exclude-ids \
    --o-filtered-table Data/tables/All_table.qza

!qiime feature-table summarize \
    --i-table Data/tables/All_table.qza \
    --o-visualization Results/tables/All_table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

Saved FeatureTable[Frequency] to: Data/tables/All_table.qza
Saved Visualization to: Results/tables/All_table.qzv


# Filtering by Treatments

In [6]:
!qiime feature-table filter-samples \
    --i-table Data/tables/All_table.qza \
    --m-metadata-file Metadata/metadata.tsv \
    --p-where "Treatment='Infected'" \
    --o-filtered-table Data/tables/Infected_table.qza

!qiime feature-table summarize \
    --i-table Data/tables/Infected_table.qza \
    --o-visualization Results/tables/Infected_table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

Saved FeatureTable[Frequency] to: Data/tables/Infected_table.qza
Saved Visualization to: Results/tables/Infected_table.qzv


# Filtering table by Niche and TimePoint

In [7]:
#Filtering by Niche
Niche_list = ['Root', 'Gall', 'J2', 'Soil']
for Niche in Niche_list:
    tabin  = 'Data/tables/Infected_table.qza'
    tabout = 'Data/tables/%s_table.qza' % Niche
    column = "Niche='%s'" % Niche     
    
    !qiime feature-table filter-samples \
        --i-table $tabin \
        --m-metadata-file Metadata/metadata.tsv \
        --p-where "$column" \
        --o-filtered-table $tabout 

    !qiime feature-table filter-features \
        --i-table $tabout \
        --p-min-frequency 3 \
        --p-min-samples 3 \
        --o-filtered-table $tabout

Saved FeatureTable[Frequency] to: Data/tables/Root_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Gall_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Gall_table.qza
Saved FeatureTable[Frequency] to: Data/tables/J2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/J2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Soil_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Soil_table.qza


In [8]:
import itertools
Niche_list = ['Root', 'Gall', 'J2', 'Soil']
for Niche_1, Niche_2 in itertools.combinations(Niche_list, 2):
    tabin  = 'Data/tables/Infected_table.qza'
    tabout = 'Data/tables/%s-%s_table.qza' % (Niche_1, Niche_2)
    column = "Niche IN ('%s','%s')"        % (Niche_1, Niche_2)
    
    !qiime feature-table filter-samples \
        --i-table $tabin \
        --m-metadata-file Metadata/metadata.tsv \
        --p-where "$column" \
        --o-filtered-table $tabout 
    !qiime feature-table filter-features \
        --i-table $tabout \
        --p-min-frequency 3 \
        --p-min-samples 3 \
        --o-filtered-table $tabout
    
    tps = [1, 2, 3, 4, 5, 6]
    if Niche_1 == 'Gall' or Niche_2 == 'Gall':
        tps = [2, 3, 4, 5, 6]
    if Niche_1 == 'J2' or Niche_2 == 'J2':
        tps = [2, 3, 4, 6]
        
    for tp in tps:
        tab_tp = 'Data/tables/%s-%s_%s_table.qza' % (Niche_1, Niche_2, tp)
        col    = "TimePoint='%s'"                 % tp
        !qiime feature-table filter-samples \
            --i-table $tabout \
            --m-metadata-file Metadata/metadata.tsv \
            --p-where "$col" \
            --o-filtered-table $tab_tp
        !qiime feature-table filter-features \
            --i-table $tab_tp \
            --p-min-frequency 2 \
            --p-min-samples 2 \
            --o-filtered-table $tab_tp

Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_5_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_5_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_6_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-Gall_6_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-J2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-J2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root-J2_2_tabl

In [9]:
import itertools
Niche_list = ['Root', 'Gall', 'J2', 'Soil']
for Niche in Niche_list:
    if Niche == 'Gall':
        tps = [2, 3, 4, 5, 6]
    if Niche == 'J2':
        tps = [2, 3, 4, 6]
    else:
        tps = [1, 2, 3, 4, 5, 6]
    for tp_1, tp_2 in itertools.combinations(tps, 2):
        tabin  = 'Data/tables/%s_table.qza'       % Niche
        tabout = 'Data/tables/%s_%s-%s_table.qza' % (Niche, tp_1, tp_2)
        column = "TimePoint IN ('%s','%s')"       % (tp_1, tp_2)

        !qiime feature-table filter-samples \
            --i-table $tabin \
            --m-metadata-file Metadata/metadata.tsv \
            --p-where "$column" \
            --o-filtered-table $tabout 

        !qiime feature-table filter-features \
            --i-table $tabout \
            --p-min-frequency 2 \
            --p-min-samples 2 \
            --o-filtered-table $tabout

Saved FeatureTable[Frequency] to: Data/tables/Root_1-2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-5_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-5_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-6_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_1-6_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_2-3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_2-3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_2-4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_2-4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_2-5_table.qza
Saved FeatureTable[Freque

In [11]:
#Filtering by TimePoint
for TP in range(1,7):
    tabin  = 'Data/tables/Infected_table.qza'
    tabout = 'Data/tables/tp-%s_table.qza' % TP
    column = "TimePoint='%s'" % TP  
    !qiime feature-table filter-samples \
        --i-table $tabin \
        --m-metadata-file Metadata/metadata.tsv \
        --p-where "$column" \
        --o-filtered-table $tabout 

    !qiime feature-table filter-features \
        --i-table $tabout \
        --p-min-frequency 2 \
        --p-min-samples 2 \
        --o-filtered-table $tabout
    
import itertools
Niche_list = ['Root', 'Gall', 'J2', 'Soil']
for Niche_1, Niche_2 in itertools.combinations(Niche_list, 2):
    for TP in range(1,7):
        tabin  = 'Data/tables/%s-%s_table.qza'       % (Niche_1, Niche_2)
        tabout = 'Data/tables/tp-%s-%s-%s_table.qza' % (TP, Niche_1, Niche_2)
        column = "TimePoint='%s'"                    % TP  
        !qiime feature-table filter-samples \
            --i-table $tabin \
            --m-metadata-file Metadata/metadata.tsv \
            --p-where "$column" \
            --o-filtered-table $tabout 

        !qiime feature-table filter-features \
            --i-table $tabout \
            --p-min-frequency 2 \
            --p-min-samples 2 \
            --o-filtered-table $tabout

Saved FeatureTable[Frequency] to: Data/tables/tp-1_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-1_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-2_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-3_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-4_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-5_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-5_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-6_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-6_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-1-Root-Gall_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-1-Root-Gall_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-2-Root-Gall_table.qza
Saved FeatureTable[Frequency] to: Data/tables/tp-2-Root

# Taxa barplot

In [4]:
import pandas as pd
outdir = 'Results/Taxa_barplots'
tabin  = 'Data/tables/Infected_table.qza'
groups = ['Group_TP','Niche']#, 'Group_Treat']
meta   = pd.read_csv('Metadata/metadata.tsv',sep='\t',index_col=0)

!mkdir -p $outdir

for group in groups:
#Collapse metadata
    meta.index = meta[group]
    meta = meta.drop_duplicates(subset=group)
    meta.index.name = '#SampleID'
    meta.to_csv('Metadata/%s_metadata.tsv'%group,sep='\t')
#Variables    
    grouped = 'Data/tables/Infected_%s_grouped.qza' % group
    taxabar = outdir + '/%s_taxabarplot.qzv'   % group
    metadata= 'Metadata/%s_metadata.tsv'       % group
#Group tables    
    !qiime feature-table group \
        --i-table $tabin \
        --p-axis 'sample' \
        --m-metadata-file Metadata/metadata.tsv \
        --m-metadata-column $group \
        --p-mode 'mean-ceiling' \
        --o-grouped-table $grouped
#Taxabarplots    
    !qiime taxa barplot \
        --i-table $grouped \
        --i-taxonomy Data/combo_taxonomy.qza \
        --m-metadata-file $metadata \
        --o-visualization $taxabar

!qiime taxa barplot \
    --i-table Data/tables/Infected_table.qza \
    --i-taxonomy Data/combo_taxonomy.qza \
    --m-metadata-file Metadata/metadata.tsv \
    --o-visualization Results/Taxa_barplots/Infected_taxabarplot.qzv

Saved FeatureTable[Frequency] to: Data/tables/Infected_Group_TP_grouped.qza
Saved Visualization to: Results/Taxa_barplots/Group_TP_taxabarplot.qzv
Saved FeatureTable[Frequency] to: Data/tables/Infected_Niche_grouped.qza
Saved Visualization to: Results/Taxa_barplots/Niche_taxabarplot.qzv
Saved Visualization to: Results/Taxa_barplots/Infected_taxabarplot.qzv


# Alpha and beta diversity analysis

In [1]:
# Alpha rarefaction plotting
outdir = 'Results/Core-metrics_analysis'
!mkdir $outdir
variants = ['Infected'] #,"All",'Root-Gall']
for variant in variants:
    depth = 15000
    alpha = outdir+'/%s_alpha-rarefaction.qzv' % variant
    tabin = 'Data/tables/%s_table.qza' % variant
    
    !qiime diversity alpha-rarefaction \
        --i-table $tabin \
        --i-phylogeny Data/rooted-tree.qza \
        --p-max-depth $depth \
        --m-metadata-file Metadata/metadata.tsv \
        --o-visualization $alpha

mkdir: cannot create directory ‘Results/Core-metrics_analysis’: File exists
Saved Visualization to: Results/Core-metrics_analysis/Infected_alpha-rarefaction.qzv


In [13]:
# core-metrics-phylogenetic: Core diversity metrics (phylogenetic and non-phylogenetic)
!mkdir Data/Core-metrics_analysis

variants = ['Root-Gall', 'Infected']
for variant in variants:
    Data_core = 'Data/Core-metrics_analysis/%s_Core-metrics'    % variant
    Rslt_core = 'Results/Core-metrics_analysis/%s_Core-metrics' % variant
    tabin     = 'Data/tables/%s_table.qza' % variant
    
    !mkdir $Rslt_core
    !qiime diversity core-metrics-phylogenetic \
        --i-phylogeny Data/rooted-tree.qza \
        --i-table $tabin \
        --p-sampling-depth 8828 \
        --m-metadata-file Metadata/metadata.tsv \
        --p-n-jobs 16 \
        --o-unweighted-unifrac-emperor $Rslt_core/unweighted-unifrac-emperor.qzv \
        --o-weighted-unifrac-emperor $Rslt_core/weighted-unifrac-emperor.qzv \
        --o-jaccard-emperor $Rslt_core/jaccard-emperor.qzv \
        --o-bray-curtis-emperor $Rslt_core/bray-curtis-emperor.qzv \
        --output-dir $Data_core

Saved FeatureTable[Frequency] to: Data/Core-metrics_analysis/All_Core-metrics/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: Data/Core-metrics_analysis/All_Core-metrics/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Data/Core-metrics_analysis/All_Core-metrics/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: Data/Core-metrics_analysis/All_Core-metrics/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Data/Core-metrics_analysis/All_Core-metrics/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: Data/Core-metrics_analysis/All_Core-metrics/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: Data/Core-metrics_analysis/All_Core-metrics/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Data/Core-metrics_analysis/All_Core-metrics/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Data/Core-metrics_analysis/All_Core-metrics/bray_curtis_distance_matrix.

In [14]:
#Principal Coordinate Analysis Biplot
variants = ['Root-Gall', 'Infected']
for variant in variants:
    Data_core = 'Data/Core-metrics_analysis/%s_Core-metrics'    % variant
    Rslt_core = 'Results/Core-metrics_analysis/%s_Core-metrics' % variant
    rel_tab   = 'Data/tables/%s_relative_table.qza'             % variant
    tabin     = 'Data/tables/%s_table.qza'                      % variant
    
#Converting feature table [Frequency] to [Relative frequency]
    !qiime feature-table relative-frequency \
        --i-table $tabin \
        --o-relative-frequency-table $rel_tab
    
#pcoa-biplot: Principal Coordinate Analysis Biplot
    artifacts = ['unweighted_unifrac', 'weighted_unifrac', 'jaccard', 'bray_curtis']
    for artifact in artifacts:
        pcoa  = Data_core+'/%s_pcoa_results.qza' % artifact
        biplot= Data_core+'/%s_biplot.qza'       % artifact
        bi_qzv= Rslt_core+'/%s_biplot.qzv'       % artifact
        
        !qiime diversity pcoa-biplot \
            --i-pcoa $pcoa \
            --i-features $rel_tab \
            --o-biplot $biplot
        !qiime emperor biplot \
            --i-biplot $biplot \
            --m-sample-metadata-file Metadata/metadata.tsv \
            --p-ignore-missing-samples \
            --p-number-of-features 5 \
            --o-visualization $bi_qzv

Saved FeatureTable[RelativeFrequency] to: Data/tables/All_relative_table.qza
Saved PCoAResults % Properties('biplot') to: Data/Core-metrics_analysis/All_Core-metrics/unweighted_unifrac_biplot.qza
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/unweighted_unifrac_biplot.qzv
Saved PCoAResults % Properties('biplot') to: Data/Core-metrics_analysis/All_Core-metrics/weighted_unifrac_biplot.qza
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/weighted_unifrac_biplot.qzv
Saved PCoAResults % Properties('biplot') to: Data/Core-metrics_analysis/All_Core-metrics/jaccard_biplot.qza
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/jaccard_biplot.qzv
Saved PCoAResults % Properties('biplot') to: Data/Core-metrics_analysis/All_Core-metrics/bray_curtis_biplot.qza
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/bray_curtis_biplot.qzv
Saved FeatureTable[RelativeFrequency] to: Data/tables/Root-Gall_relative_table.qza


In [15]:
variants = ['Root-Gall', 'Infected']
for variant in variants:
    core   = 'Data/Core-metrics_analysis/%s_Core-metrics'    % variant
    outdir = 'Results/Core-metrics_analysis/%s_Core-metrics' % variant

#Alpha diversity comparisons¶    
    alpha_diversity = ['shannon', 'faith_pd', 'evenness', 'observed_otus']
    for alpha in alpha_diversity:
        vector  = core+'/%s_vector.qza'                      % alpha
        ags_qzv = outdir+'/%s_vector_alpha-group-sign.qzv'   % alpha
        
        !qiime diversity alpha-group-significance \
            --i-alpha-diversity $vector \
            --m-metadata-file Metadata/metadata.tsv \
            --o-visualization $ags_qzv

Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/shannon_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/faith_pd_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/evenness_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/observed_otus_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/Root-Gall_Core-metrics/shannon_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/Root-Gall_Core-metrics/faith_pd_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/Root-Gall_Core-metrics/evenness_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/Root-Gall_Core-metrics/observed_otus_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis/Infected_Core-metrics/shannon_vector_alpha-group-s

In [16]:
#Beta diversity comparisons¶
variants = ['Root-Gall', 'Infected']
for variant in variants:
    core   = 'Data/Core-metrics_analysis/%s_Core-metrics'    % variant
    outdir = 'Results/Core-metrics_analysis/%s_Core-metrics' % variant
    beta_diversity = ['unweighted_unifrac', 'weighted_unifrac', 'jaccard', 'bray_curtis']
    for beta in beta_diversity:
        distance= core+'/%s_distance_matrix.qza' % beta
        columns = ['Group_TP', 'Group_Treat']
        for column in columns:
            bgs_qzv = outdir+'/%s_%s-dist-matrix_beta-group-sign-pairwise.qzv' % (beta, column)
            
            !qiime diversity beta-group-significance \
                --i-distance-matrix $distance \
                --m-metadata-file Metadata/metadata.tsv \
                --m-metadata-column $column \
                --p-pairwise \
                --o-visualization $bgs_qzv         

Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/unweighted_unifrac_Group_TP-dist-matrix_beta-group-sign-pairwise.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/unweighted_unifrac_Group_Treat-dist-matrix_beta-group-sign-pairwise.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/weighted_unifrac_Group_TP-dist-matrix_beta-group-sign-pairwise.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/weighted_unifrac_Group_Treat-dist-matrix_beta-group-sign-pairwise.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/jaccard_Group_TP-dist-matrix_beta-group-sign-pairwise.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/jaccard_Group_Treat-dist-matrix_beta-group-sign-pairwise.qzv
Saved Visualization to: Results/Core-metrics_analysis/All_Core-metrics/bray_curtis_Group_TP-dist-matrix_beta-group-sign-pairwise.qzv
Saved Visualization to: Results/Core-metrics

# Core features

In [1]:
#By Niches and TimePoints
outdir= 'Results/Core_Features'
groups = ['S_2','S_3','S_4','S_6',
          'R_2','R_3','R_4','R_6',
          'G_2','G_3','G_4','G_6',
          'J_2','J_3','J_4','J_6']

!mkdir $outdir
for group in groups:
    tabin  = 'Data/tables/Infected_table.qza'
    column = "Group_TP='%s'"            % group
    tabout = 'Data/tables/%s_table.qza' % group

    !qiime feature-table filter-samples \
        --i-table $tabin \
        --m-metadata-file Metadata/metadata.tsv \
        --p-where "$column" \
        --o-filtered-table $tabout
     
    table_7= 'Data/tables/%s_7-lev_table.qza'  % group
    core_7 = outdir+'/%s_7-lev_core.qzv'       % group
    coreqzv= outdir+'/%s_core.qzv'             % group

    !qiime taxa collapse \
        --i-table $tabout \
        --i-taxonomy Data/combo_taxonomy.qza \
        --p-level 7 \
        --o-collapsed-table $table_7
    !qiime feature-table core-features \
        --i-table $table_7 \
        --o-visualization $core_7
    !qiime feature-table core-features \
        --i-table $tabout \
        --o-visualization $coreqzv

Saved FeatureTable[Frequency] to: Data/tables/S_2_table.qza
mkdir: missing operand
Try 'mkdir --help' for more information.
Saved FeatureTable[Frequency] to: Data/tables/S_2_7-lev_table.qza
Saved Visualization to: Results/Core_Features/S_2_7-lev_core.qzv
Saved Visualization to: Results/Core_Features/S_2_core.qzv
Saved FeatureTable[Frequency] to: Data/tables/S_3_table.qza
mkdir: missing operand
Try 'mkdir --help' for more information.
Saved FeatureTable[Frequency] to: Data/tables/S_3_7-lev_table.qza
Saved Visualization to: Results/Core_Features/S_3_7-lev_core.qzv
Saved Visualization to: Results/Core_Features/S_3_core.qzv
Saved FeatureTable[Frequency] to: Data/tables/S_4_table.qza
mkdir: missing operand
Try 'mkdir --help' for more information.
Saved FeatureTable[Frequency] to: Data/tables/S_4_7-lev_table.qza
Saved Visualization to: Results/Core_Features/S_4_7-lev_core.qzv
Saved Visualization to: Results/Core_Features/S_4_core.qzv
Saved FeatureTable[Frequency] to: Data/tables/S_6_table.qz

# Core-features_tables

In [ ]:
# By Niches and TimePoints separately 
import pandas as pd
outdir= 'Results/Core_Features'
groups = ['S_2','S_3','S_4','S_6',
          'R_2','R_3','R_4','R_6',
          'G_2','G_3','G_4','G_6',
          'J_2','J_3','J_4','J_6']

for group in groups:
    temp = 'Data/Core_%s_temp' % group
    coreqzv = 'Results/Core_Features/%s_core.qzv' % group
    !mkdir $temp
    !qiime tools export \
        --input-path $coreqzv \
        --output-path $temp

    temp_7 = 'Data/Core_%s_temp_7' % group
    core_7 = 'Results/Core_Features/%s_7-lev_core.qzv' % group
    !mkdir $temp_7
    !qiime tools export \
        --input-path $core_7 \
        --output-path $temp_7

Root, Gall, Soil, J2 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for group in groups:
    temp = 'Data/Core_%s_temp' % group
    core_tsv= temp+'/core-features-1.000.tsv' 
    core_pd = pd.read_csv(core_tsv, sep='\t')
    core_pd = core_pd.filter(['Feature ID'], axis = 1)
    core_pd.columns = [group]
    if 'S' in group: Soil = pd.concat([Soil, core_pd], axis = 1)
    if 'R' in group: Root = pd.concat([Root, core_pd], axis = 1)
    if 'G' in group: Gall = pd.concat([Gall, core_pd], axis = 1)
    if 'J' in group: J2   = pd.concat([J2, core_pd], axis = 1)
    !rm -r $temp
        
Soil.to_csv('Results/Core_Features/Soil_tp.tsv', sep='\t')
Root.to_csv('Results/Core_Features/Root_tp.tsv', sep='\t')
Gall.to_csv('Results/Core_Features/Gall_tp.tsv', sep='\t')
J2.to_csv('Results/Core_Features/J2_tp.tsv', sep='\t')

Root, Gall, Soil, J2 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for group in groups:
    temp_7 = 'Data/Core_%s_temp_7' % group
    core_tsv= temp_7+'/core-features-1.000.tsv' 
    core_pd = pd.read_csv(core_tsv, sep='\t')
    core_pd = core_pd.filter(['Feature ID'], axis = 1)
    core_pd.columns = [group]
    if 'S' in group: Soil = pd.concat([Soil, core_pd], axis = 1)
    if 'R' in group: Root = pd.concat([Root, core_pd], axis = 1)
    if 'G' in group: Gall = pd.concat([Gall, core_pd], axis = 1)
    if 'J' in group: J2   = pd.concat([J2, core_pd], axis = 1)
    !rm -r $temp_7
        
Soil.to_csv('Results/Core_Features/Soil_tp_lev-7.tsv', sep='\t')
Root.to_csv('Results/Core_Features/Root_tp_lev-7.tsv', sep='\t')
Gall.to_csv('Results/Core_Features/Gall_tp_lev-7.tsv', sep='\t')
J2.to_csv('Results/Core_Features/J2_tp_lev-7.tsv', sep='\t')

# LONGITUDINAL

### Pairwise differences & distances comparisons

In [1]:
# Differences
import itertools
timepoints = [2,3,4,6]
variants = ['Infected', 'Root-Gall']
for variant in variants:
    core_dt = 'Data/Core-metrics_analysis/%s_Core-metrics' % variant
    core_rs = 'Results/Longitudinal/%s'                    % variant
    tabin = 'Data/tables/%s_table.qza'                     % variant
    
    !mkdir -p $core_dt $core_rs/Pairwise-differences
    for state, state_2 in itertools.combinations(timepoints, 2):
        alpha_diversity = ['shannon', 'faith_pd', 'evenness', 'observed_otus']
        for alpha in alpha_diversity:
            vector=core_dt+'/%s_vector.qza' % alpha
            param =core_rs+'/Pairwise-differences/%s_%s_pairwise-differences_parametric_%s-%s.qzv'   %(variant,alpha,state,state_2)
            no_par=core_rs+'/Pairwise-differences/%s_%s_pairwise-differences_no-parametric_%s-%s.qzv'%(variant,alpha,state,state_2)
            if alpha == 'evenness':
                metric = 'pielou_e'
            else:
                metric = alpha
            !qiime longitudinal pairwise-differences \
                --m-metadata-file Metadata/metadata.tsv \
                --m-metadata-file $vector \
                --p-metric $metric \
                --p-group-column Niche \
                --p-state-column TimePoint \
                --p-state-1 $state \
                --p-state-2 $state_2 \
                --p-individual-id-column Location \
                --p-replicate-handling random \
                --p-no-parametric \
                --o-visualization $no_par
            !qiime longitudinal pairwise-differences \
                --m-metadata-file Metadata/metadata.tsv \
                --m-metadata-file $vector \
                --p-metric $metric \
                --p-group-column Niche \
                --p-state-column TimePoint \
                --p-state-1 $state \
                --p-state-2 $state_2 \
                --p-individual-id-column Location \
                --p-replicate-handling random \
                --p-parametric \
                --o-visualization $param   

Saved Visualization to: Results/Longitudinal/Infected/Pairwise-differences/Infected_shannon_pairwise-differences_no-parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-differences/Infected_shannon_pairwise-differences_parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-differences/Infected_faith_pd_pairwise-differences_no-parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-differences/Infected_faith_pd_pairwise-differences_parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-differences/Infected_evenness_pairwise-differences_no-parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-differences/Infected_evenness_pairwise-differences_parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-differences/Infected_observed_otus_pairwise-differences_no-parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/

In [2]:
#Distances
import itertools
timepoints = [2,3,4,6]
variants = ['Infected', 'Root-Gall']
for variant in variants:
    core_dt = 'Data/Core-metrics_analysis/%s_Core-metrics' % variant
    core_rs = 'Results/Longitudinal/%s'                    % variant
    tabin = 'Data/tables/%s_table.qza'                     % variant
    
    !mkdir $core_rs/Pairwise-distances
    beta_diversity = ['unweighted_unifrac', 'weighted_unifrac', 'jaccard', 'bray_curtis']
    for beta in beta_diversity:
        for state, state_2 in itertools.combinations(timepoints, 2):
            distance= core_dt+'/%s_distance_matrix.qza' % beta
            output  = core_rs+'/Pairwise-distances/%s_%s_pairwise-distances_no-parametric_%s-%s.qzv' % (variant, beta, state, state_2)
            param   = core_rs+'/Pairwise-distances/%s_%s_pairwise-distances_parametric_%s-%s.qzv'    % (variant, beta, state, state_2)
            
            !qiime longitudinal pairwise-distances \
                --i-distance-matrix $distance \
                --m-metadata-file Metadata/metadata.tsv \
                --p-group-column Niche \
                --p-state-column TimePoint \
                --p-state-1 $state \
                --p-state-2 $state_2 \
                --p-individual-id-column Location \
                --p-replicate-handling random \
                --o-visualization $output
            !qiime longitudinal pairwise-distances \
                --i-distance-matrix $distance \
                --m-metadata-file Metadata/metadata.tsv \
                --p-group-column Niche \
                --p-state-column TimePoint \
                --p-state-1 $state \
                --p-state-2 $state_2 \
                --p-individual-id-column Location \
                --p-replicate-handling random \
                --p-parametric \
                --o-visualization $param

mkdir: cannot create directory ‘Results/Longitudinal/Infected/Pairwise-distances’: File exists
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-distances/Infected_unweighted_unifrac_pairwise-distances_no-parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-distances/Infected_unweighted_unifrac_pairwise-distances_parametric_2-3.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-distances/Infected_unweighted_unifrac_pairwise-distances_no-parametric_2-4.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-distances/Infected_unweighted_unifrac_pairwise-distances_parametric_2-4.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-distances/Infected_unweighted_unifrac_pairwise-distances_no-parametric_2-6.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise-distances/Infected_unweighted_unifrac_pairwise-distances_parametric_2-6.qzv
Saved Visualization to: Results/Longitudinal/Infected/Pairwise

### Feature volatility analysis

In [7]:
#By_Niches
niches = ['Gall','Root','J2','Soil']
levels = ['Taxa','ASV']
for level in levels:
    for niche in niches:
        out_rs = 'Results/Longitudinal/By_Niches/%s_%s_Feature_Volatility' %(niche,level)
        out_dt = 'Data/Longitudinal/By_Niches/%s_%s_Feature_Volatility'    %(niche,level)
        volplot= out_rs + '/%s_%s_volatility-plot.qzv'                     %(niche,level)
        tabin  = 'Data/tables/%s_table.qza'                                %niche
        tabin7 = 'Data/tables/%s_table_7_all_tp.qza'                       %niche
        if level == 'Taxa':
            table = tabin7
            !qiime taxa collapse \
                --i-table $tabin \
                --i-taxonomy Data/combo_taxonomy.qza \
                --p-level 7 \
                --o-collapsed-table $tabin7
        else:
            table = tabin
        !mkdir -p $out_rs 
        !qiime longitudinal feature-volatility \
            --i-table $table \
            --m-metadata-file Metadata/metadata.tsv \
            --p-state-column TimePoint \
            --p-individual-id-column Location \
            --output-dir $out_dt \
            --p-n-jobs 16 \
            --p-feature-count 'all' \
            --o-volatility-plot $volplot
#!rm Data/Longitudinal/By_Niches

Saved FeatureTable[Frequency] to: Data/tables/Gall_table_7_all_tp.qza
Saved FeatureTable[RelativeFrequency] to: Data/Longitudinal/By_Niches/Gall_Taxa_Feature_Volatility/filtered_table.qza
Saved FeatureData[Importance] to: Data/Longitudinal/By_Niches/Gall_Taxa_Feature_Volatility/feature_importance.qza
Saved Visualization to: Results/Longitudinal/By_Niches/Gall_Taxa_Feature_Volatility/Gall_Taxa_volatility-plot.qzv
Saved Visualization to: Data/Longitudinal/By_Niches/Gall_Taxa_Feature_Volatility/accuracy_results.qzv
Saved SampleEstimator[Regressor] to: Data/Longitudinal/By_Niches/Gall_Taxa_Feature_Volatility/sample_estimator.qza
Saved FeatureTable[Frequency] to: Data/tables/Root_table_7_all_tp.qza
Saved FeatureTable[RelativeFrequency] to: Data/Longitudinal/By_Niches/Root_Taxa_Feature_Volatility/filtered_table.qza
Saved FeatureData[Importance] to: Data/Longitudinal/By_Niches/Root_Taxa_Feature_Volatility/feature_importance.qza
Saved Visualization to: Results/Longitudinal/By_Niches/Root_Taxa_

In [8]:
variants = ['Infected', 'Root-Gall']
for variant in variants:
    tabin  = 'Data/tables/%s_table.qza'                   % variant
    out_rs = 'Results/Longitudinal/%s/Feature_Volatility' % variant
    out_dt = 'Data/Longitudinal/%s/Feature_Volatility'    % variant
    volplot= out_rs + '/%s_volatility-plot.qzv'           % variant
    
    !mkdir -p $out_rs 
    !qiime longitudinal feature-volatility \
        --i-table $tabin \
        --m-metadata-file Metadata/metadata.tsv \
        --p-state-column TimePoint \
        --p-individual-id-column Location \
        --output-dir $out_dt \
        --p-n-jobs 16 \
        --p-feature-count 'all' \
        --o-volatility-plot $volplot
    
   
    out_rs_level = 'Results/Longitudinal/%s/Feature_Volatility_lev-7' % variant
    out_dt_level = 'Data/Longitudinal/%s/Feature_Volatility_lev-7'    % variant
    tabout_level = 'Data/Longitudinal/%s_7_table.qza'                 % variant
    volplot_level= out_rs_level + '/%s_volatility-plot_7-lev.qzv'     % variant
    
    !mkdir -p $out_rs_level  
    !qiime taxa collapse \
        --i-table $tabin \
        --i-taxonomy Data/combo_taxonomy.qza \
        --p-level 7 \
        --o-collapsed-table $tabout_level
    !qiime longitudinal feature-volatility \
        --i-table $tabout_level \
        --m-metadata-file Metadata/metadata.tsv \
        --p-state-column TimePoint \
        --p-individual-id-column Location \
        --p-n-jobs 16 \
        --p-feature-count 'all' \
        --output-dir $out_dt_level \
        --o-volatility-plot $volplot_level
#!rm Data/Longitudinal

Saved FeatureTable[RelativeFrequency] to: Data/Longitudinal/Infected/Feature_Volatility/filtered_table.qza
Saved FeatureData[Importance] to: Data/Longitudinal/Infected/Feature_Volatility/feature_importance.qza
Saved Visualization to: Results/Longitudinal/Infected/Feature_Volatility/Infected_volatility-plot.qzv
Saved Visualization to: Data/Longitudinal/Infected/Feature_Volatility/accuracy_results.qzv
Saved SampleEstimator[Regressor] to: Data/Longitudinal/Infected/Feature_Volatility/sample_estimator.qza
Saved FeatureTable[Frequency] to: Data/Longitudinal/Infected_7_table.qza
Saved FeatureTable[RelativeFrequency] to: Data/Longitudinal/Infected/Feature_Volatility_lev-7/filtered_table.qza
Saved FeatureData[Importance] to: Data/Longitudinal/Infected/Feature_Volatility_lev-7/feature_importance.qza
Saved Visualization to: Results/Longitudinal/Infected/Feature_Volatility_lev-7/Infected_volatility-plot_7-lev.qzv
Saved Visualization to: Data/Longitudinal/Infected/Feature_Volatility_lev-7/accuracy

# 97% OTU Clustering and core metrics

In [1]:
!mkdir Data/97_OTU
!qiime vsearch cluster-features-de-novo \
    --i-table Data/tables/table.qza \
    --i-sequences Data/rep-seqs.qza \
    --p-perc-identity 0.97 \
    --o-clustered-table Data/97_OTU/table_97-OTU.qza \
    --o-clustered-sequences Data/97_OTU/rep-seqs_97.qza

!qiime feature-table summarize \
    --i-table Data/97_OTU/table_97-OTU.qza  \
    --o-visualization Data/97_OTU/table_97-OTU.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

mkdir: cannot create directory ‘Data/97_OTU’: File exists
Saved FeatureTable[Frequency] to: Data/97_OTU/table_97-OTU.qza
Saved FeatureData[Sequence] to: Data/97_OTU/rep-seqs_97.qza
Saved Visualization to: Data/97_OTU/table_97-OTU.qzv


In [2]:
!qiime feature-table filter-features \
    --i-table Data/97_OTU/table_97-OTU.qza \
    --p-min-frequency 10 \
    --p-min-samples 5 \
    --o-filtered-table Data/97_OTU/filtered-table.qza

!qiime taxa filter-table \
    --i-table Data/97_OTU/filtered-table.qza \
    --i-taxonomy Data/taxonomy.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table Data/97_OTU/filtered-table.qza

!qiime feature-table filter-samples \
    --i-table Data/97_OTU/filtered-table.qza \
    --p-min-features 25 \
    --p-min-frequency 8000 \
    --o-filtered-table Data/97_OTU/filtered-table.qza

!qiime feature-table filter-samples \
    --i-table Data/97_OTU/filtered-table.qza \
    --m-metadata-file Data/tables/samples_to_exclude.tsv \
    --p-exclude-ids \
    --o-filtered-table Data/97_OTU/filtered-table.qza

!qiime feature-table summarize \
    --i-table Data/97_OTU/filtered-table.qza \
    --o-visualization Data/97_OTU/filtered-table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

!qiime taxa filter-table \
    --i-table Data/97_OTU/filtered-table.qza \
    --i-taxonomy Data/taxonomy.qza \
    --p-include D_1__ \
    --o-filtered-table Data/97_OTU/filtered-table.qza

!qiime feature-table summarize \
    --i-table Data/97_OTU/filtered-table.qza \
    --o-visualization Data/97_OTU/filtered-table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

Saved FeatureTable[Frequency] to: Data/97_OTU/filtered-table.qza
Saved FeatureTable[Frequency] to: Data/97_OTU/filtered-table.qza
Saved FeatureTable[Frequency] to: Data/97_OTU/filtered-table.qza
Saved FeatureTable[Frequency] to: Data/97_OTU/filtered-table.qza
Saved Visualization to: Data/97_OTU/filtered-table.qzv
Saved FeatureTable[Frequency] to: Data/97_OTU/filtered-table.qza
Saved Visualization to: Data/97_OTU/filtered-table.qzv


In [3]:
!qiime feature-table filter-samples \
    --i-table Data/97_OTU/filtered-table.qza \
    --m-metadata-file Metadata/metadata.tsv \
    --p-where "Treatment='Infected'" \
    --o-filtered-table Data/97_OTU/Infected_table.qza
!qiime feature-table summarize \
    --i-table Data/97_OTU/Infected_table.qza \
    --o-visualization Data/97_OTU/Infected_table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv
    
!qiime feature-table filter-samples \
    --i-table Data/97_OTU/Infected_table.qza \
    --m-metadata-file Metadata/metadata.tsv \
    --p-where "Niche IN ('Root','Gall')" \
    --o-filtered-table Data/97_OTU/Root-Gall_table.qza
!qiime feature-table summarize \
    --i-table Data/97_OTU/Root-Gall_table.qza \
    --o-visualization Data/97_OTU/Root-Gall_table.qzv \
    --m-sample-metadata-file Metadata/metadata.tsv

Saved FeatureTable[Frequency] to: Data/97_OTU/Infected_table.qza
Saved Visualization to: Data/97_OTU/Infected_table.qzv
Saved FeatureTable[Frequency] to: Data/97_OTU/Root-Gall_table.qza
Saved Visualization to: Data/97_OTU/Root-Gall_table.qzv


In [17]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences Data/97_OTU/rep-seqs_97.qza \
    --o-alignment Data/97_OTU/aligned-rep-seqs.qza \
    --o-masked-alignment Data/97_OTU/masked-aligned-rep-seqs.qza \
    --o-tree Data/97_OTU/unrooted-tree.qza \
    --o-rooted-tree Data/97_OTU/rooted-tree_97-OTU.qza \
    --p-n-threads 4

Saved FeatureData[AlignedSequence] to: Data/97_OTU/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: Data/97_OTU/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: Data/97_OTU/unrooted-tree.qza
Saved Phylogeny[Rooted] to: Data/97_OTU/rooted-tree_97-OTU.qza


In [4]:
!mkdir Results/Core-metrics_analysis_97-OTU

variants = ['Root-Gall', 'Infected']
for variant in variants:
    Rslt_core = 'Results/Core-metrics_analysis_97-OTU/%s_Core-metrics' % variant
    tabin     = 'Data/97_OTU/%s_table.qza'    % variant

    !qiime diversity core-metrics-phylogenetic \
        --i-phylogeny Data/97_OTU/rooted-tree_97-OTU.qza \
        --i-table $tabin \
        --p-sampling-depth 8867 \
        --m-metadata-file Metadata/metadata.tsv \
        --p-n-jobs 4 \
        --output-dir $Rslt_core

Saved FeatureTable[Frequency] to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metri

In [5]:
variants = ['Root-Gall', 'Infected']
for variant in variants:
    core   = 'Results/Core-metrics_analysis_97-OTU/%s_Core-metrics' % variant
#Alpha diversity comparisons¶    
    alpha_diversity = ['shannon', 'faith_pd', 'evenness', 'observed_otus']
    for alpha in alpha_diversity:
        vector  = core+'/%s_vector.qza'                    % alpha
        ags_qzv = core+'/%s_vector_alpha-group-sign.qzv'   % alpha
        
        !qiime diversity alpha-group-significance \
            --i-alpha-diversity $vector \
            --m-metadata-file Metadata/metadata.tsv \
            --o-visualization $ags_qzv

Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/shannon_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/faith_pd_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/evenness_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Root-Gall_Core-metrics/observed_otus_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Infected_Core-metrics/shannon_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Infected_Core-metrics/faith_pd_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Infected_Core-metrics/evenness_vector_alpha-group-sign.qzv
Saved Visualization to: Results/Core-metrics_analysis_97-OTU/Infected_Core-metrics/observed_otus_vector_alpha-group-sign.qzv
